In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff




In [2]:
yandex_world = pd.read_csv('yandex_world.csv')
yandex_world.rename(columns = {'Дата':'Date','Страна':'Country','Заражений':'Confirmed','Выздоровлений':'Recovered',
                               'Смертей':'Deaths','Смертей за день':'Day_deaths',
                               'Заражений за день':'Day_confirmed','Выздоровлений за день':'Day_recovered'},inplace = True)
for i in range(yandex_world.shape[0]):
    a= yandex_world.Date[i].split('.')
    yandex_world.Date[i]=datetime.date(int(a[2]),int(a[1]),int(a[0]))

yandex_world.tail(20)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
39998,Япония,2020-08-03,40099,26476,1018,983,728,5
39999,Япония,2020-08-04,41347,27187,1023,1248,711,5
40000,Япония,2020-08-05,42686,28020,1028,1339,833,5
40001,Япония,2020-08-06,44167,28858,1034,1481,838,6
40002,Япония,2020-08-07,45764,30241,1042,1597,1383,8
40003,Япония,2020-08-08,47342,31248,1042,1578,1007,0
40004,Япония,2020-08-09,48782,32867,1047,1440,1619,5
40005,Япония,2020-08-10,49617,33450,1052,835,583,5
40006,Япония,2020-08-11,50302,34136,1058,685,686,6
40007,Япония,2020-08-12,51288,34969,1066,986,833,8


In [3]:
df3_temp = yandex_world.groupby('Date').sum()
df3_temp['Country']='Весь мир'

y5= pd.concat([yandex_world,df3_temp.reset_index()],axis = 0)
y5.reset_index()
yandex_world = y5
yandex_world.tail(10)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
204,Весь мир,2020-08-13,20907124,12992176,750230,285984,165361,6241
205,Весь мир,2020-08-14,21211573,13276831,760374,304449,284655,10144
206,Весь мир,2020-08-15,21459699,13445842,765718,248126,169011,5344
207,Весь мир,2020-08-16,21672186,13676868,769904,212487,231026,4186
208,Весь мир,2020-08-17,21881858,13888301,774034,209672,211433,4130
209,Весь мир,2020-08-18,22136954,14116451,780908,255096,228150,6874
210,Весь мир,2020-08-19,22411300,14333914,787672,274346,217463,6764
211,Весь мир,2020-08-20,22678483,14541573,793698,267183,207659,6026
212,Весь мир,2020-08-21,22949234,14712252,799252,270751,170679,5554
213,Весь мир,2020-08-22,23203532,14911319,804416,254298,199067,5164


In [4]:
# dt = datetime.datetime.today()
# today = datetime.date(dt.year,dt.month,dt.day)
# ystday = datetime.date(today.year,today.month,today.day-1)
# yst2day = datetime.date(today.year,today.month,today.day-2)
# yst3day = datetime.date(today.year,today.month,today.day-3)

# print(today)
# print(ystday)
# print(yst2day)
# print(yst3day)
myday = list(yandex_world.tail(1).Date)[0]

print(yandex_world.iloc[-1,0])
print('Выбран день: ',myday)

Весь мир
Выбран день:  2020-08-22


In [5]:
X = pd.DataFrame()
country_dict = {}
yandex_world['Rt'] = 1
yandex_world['Rt2'] = 1
yandex_world['Rate_conf'] = 0
yandex_world['Rate_ill'] = 0

temp_df = yandex_world.groupby(['Country','Date']).sum()

# Ниже enumerate  для составления словаря из имен регионов {i:j} , для расчетов можно было использовать только j

for i,j in enumerate(temp_df.index.get_level_values(0).unique()):
    temp_df2 =  temp_df.loc[j]
    temp1 = temp_df2['Day_confirmed'].rolling(4).sum()/(temp_df2["Day_confirmed"].rolling(8).sum()-

                                                              temp_df2["Day_confirmed"].rolling(4).sum())
    temp2 = temp_df2['Confirmed'].rolling(4).sum()/(temp_df2['Confirmed'].rolling(8).sum()-

                                                              temp_df2['Confirmed'].rolling(4).sum())
    temp_df2['Rt'] = round(temp1,3)
    temp_df2['Rt2'] = round(temp2,3)
    temp_df2['Country'] = j
    temp_df2['Remaining_ill']=temp_df2['Confirmed']-temp_df2['Recovered']-temp_df2['Deaths']
    temp_df2['MA7_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=7).mean()
    temp_df2['MA14_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=14).mean()
    temp_df2['MA7_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=7).mean()
    temp_df2['MA14_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=14).mean()
    
    temp_df2['Change_dayconf'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_dayconf'].iloc[k]=temp_df2['Day_confirmed'].iloc[k]-temp_df2['Day_confirmed'].iloc[k-1]
    temp_df2['Change_remill'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_remill'].iloc[k]=temp_df2['Remaining_ill'].iloc[k]-temp_df2['Remaining_ill'].iloc[k-1]
    
    

   #Генератор DF  на каждый регион(сидит в цикле):
   # globals()['x' + str(i)] = temp_df2.reset_index()
    
    X = pd.concat([X,temp_df2.reset_index()],axis = 0)
    country_dict[i]=j
    
print(country_dict.items())
    
X = X.reset_index(drop=True)
X.tail()


dict_items([(0, 'Австралия'), (1, 'Австрия'), (2, 'Азербайджан'), (3, 'Албания'), (4, 'Алжир'), (5, 'Ангола'), (6, 'Андорра'), (7, 'Антигуа и Барбуда'), (8, 'Аргентина'), (9, 'Армения'), (10, 'Афганистан'), (11, 'Багамские острова'), (12, 'Бангладеш'), (13, 'Барбадос'), (14, 'Бахрейн'), (15, 'Беларусь'), (16, 'Белиз'), (17, 'Бельгия'), (18, 'Бенин'), (19, 'Болгария'), (20, 'Боливия'), (21, 'Босния и Герцеговина'), (22, 'Ботсвана'), (23, 'Бразилия'), (24, 'Бруней'), (25, 'Буркина-Фасо'), (26, 'Бурунди'), (27, 'Бутан'), (28, 'Великобритания'), (29, 'Венгрия'), (30, 'Венесуэла'), (31, 'Весь мир'), (32, 'Восточный Тимор'), (33, 'Вьетнам'), (34, 'Габон'), (35, 'Гаити'), (36, 'Гайана'), (37, 'Гамбия'), (38, 'Гана'), (39, 'Гватемала'), (40, 'Гвинея'), (41, 'Гвинея-Бисау'), (42, 'Германия'), (43, 'Гондурас'), (44, 'Гренада'), (45, 'Греция'), (46, 'Грузия'), (47, 'ДР Конго'), (48, 'Дания'), (49, 'Джибути'), (50, 'Доминика'), (51, 'Доминиканская Республика'), (52, 'Египет'), (53, 'Замбия'), (54,

,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths,Rt,Rt2,Rate_conf,Rate_ill,Country,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
40227,2020-08-18,57636,42965,1135,919,1374,23,0.909,1.085,0,0,Япония,13536,1047.714286,1163.500000,14882.857143,14765.500000,276,-478
40228,2020-08-19,58728,43786,1148,1092,821,13,0.774,1.078,0,0,Япония,13794,1062.857143,1145.857143,14674.428571,14776.642857,173,258
40229,2020-08-20,59900,45110,1157,1172,1324,9,0.799,1.072,0,0,Япония,13633,1061.285714,1123.785714,14449.571429,14730.785714,80,-161
40230,2020-08-21,60949,46264,1175,1049,1154,18,0.997,1.070,0,0,Япония,13510,1018.714286,1084.642857,13916.714286,14661.428571,-123,-123
40231,2020-08-22,61916,47418,1179,967,1154,4,1.121,1.071,0,0,Япония,13319,980.714286,1041.000000,13745.285714,14537.642857,-82,-191


In [6]:
# Добавляем колонки расчитанные сколько доля их в процентах от суммарного показателя в мире
mask = X.Country == 'Весь мир'
temp_df = X[mask][['Date','Confirmed','Remaining_ill']]
temp_df.index = temp_df.Date
temp_df.drop(columns =['Date'], inplace = True)
print(temp_df.tail(2))
# print(temp_df.index.get_level_values(0))
# temp_df.loc[datetime.date(2020,6,15),'Россия'].Confirmed
# for i in temp_df.index.get_level_values(0).unique()[-2:]:
#     print(temp_df.loc[i,'Россия'].Confirmed)
world_date = list(temp_df.index)
world_conf = list(temp_df.Confirmed)
world_ill = list(temp_df.Remaining_ill)
world_dict_conf = dict(zip(world_date,world_conf))
world_dict_ill = dict(zip(world_date,world_ill))

print(len(X))
print(X.shape)
print(X.loc[0,'Date'])
for i in range(len(X)):
    X.loc[i,'Rate_conf']=100*X.loc[i,'Confirmed']/world_dict_conf.get(X.loc[i,'Date'])
    X.loc[i,'Rate_ill']=100*X.loc[i,'Remaining_ill']/world_dict_ill.get(X.loc[i,'Date'])


Confirmed  Remaining_ill
Date                                
2020-08-21   22949234        7437730
2020-08-22   23203532        7487797
40232
(40232, 19)
2020-01-22


In [7]:
# Проверка расчетов процентных показателей
mask = (X.Date>=datetime.date(2020,6,13))&(X.Country.isin(['Весь мир','Россия']))

temp=X[mask]
temp[['Date','Confirmed','Country','Rate_conf','Rate_ill']].tail(20)

,Date,Confirmed,Country,Rate_conf,Rate_ill
27586,2020-08-03,854641,Россия,4.674714,2.815737
27587,2020-08-04,859762,Россия,4.637138,2.760179
27588,2020-08-05,864948,Россия,4.597864,2.705921
27589,2020-08-06,870187,Россия,4.556633,2.638702
27590,2020-08-07,875378,Россия,4.517372,2.576565
27591,2020-08-08,880563,Россия,4.484088,2.535220
27592,2020-08-09,885718,Россия,4.459431,2.546444
27593,2020-08-10,890799,Россия,4.434125,2.550231
27594,2020-08-11,895691,Россия,4.402687,2.546410
27595,2020-08-12,900745,Россия,4.368066,2.505044


In [8]:
X.to_csv('W.csv',index=False)

In [9]:
covid_rates = pd.DataFrame()
mask = (X.Date==myday)&(X.Country!='Весь мир')
today_cases = X[mask]
#covid_rates.reset_index(drop=True)
covid_rates['По количеству текущих больных'] = today_cases.sort_values(by=['Remaining_ill'],ascending = False).Country.reset_index(drop=True)
covid_rates['По количеству новых'] = today_cases.sort_values(by=['Day_confirmed'],ascending = False).Country.reset_index(drop=True)
covid_rates['По умершим за день'] = today_cases.sort_values(by=['Day_deaths'],ascending = False).Country.reset_index(drop=True)
covid_rates['По выздоровевшим за день'] = today_cases.sort_values(by=['Day_recovered'],ascending = False).Country.reset_index(drop=True)
covid_rates.head(10)
fig =  ff.create_table(covid_rates.head(15))
fig.update_layout(
    
                  title_text ='Таблица рейтингов заболеваемости COVID-19 по странам',
                  margin = {'t':100, 'b':100},
                  title_x = 0.5,
                  title_y= 0.95,
                  title_xanchor = "center",
                  title_yanchor = "middle", 
                  
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ''
)


fig.show()


In [10]:
country_number = 5
country_towatch = list(covid_rates['По количеству текущих больных'].head(country_number))
#country_towatch.append('Турция')
#сountry_towatch.append('Танзания')
country_towatch

['США', 'Индия', 'Бразилия', 'Великобритания', 'Испания']

In [11]:
mask = (X.Date>datetime.date(2020,3,1))&(X.Country.isin(country_towatch))
#mask = (X.Region.isin(regions_towatch))

fig = px.line(X[mask], y="Day_confirmed", x="Date",
                             color="Country", line_group="Country",
              line_shape="spline", render_mode="notebook",labels={'Country':'Страна:'},
              color_discrete_map={
                "США": "red",
                "Бразилия": "magenta",
                "Россия": "goldenrod",
                "Индия": "darkblue",
                "Великобритания": "green",
                 'Испания': 'aquamarine'},
             title="График количества новых заболевших COVID-19 по 5 странам<br> с наибольшим количеством текущих больных")

fig.update_layout(
    annotations=[
            dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Данные: Яндекс",
            xref="paper",
            yref="paper")],
           
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ''
)


fig.show()

In [12]:
#mask = (X.Date>datetime.date(2020,5,1))&(X.Region.isin(regions_towatch))

colors_dict = { "США": "red",
                "Бразилия": "magenta",
                "Россия": "goldenrod",
                "Индия": "darkblue",
                "Великобритания": "green",
                 'Испания': 'aquamarine'}
fig = go.Figure()
for i in country_towatch:
    mask = X['Country'] == i
        
    fig.add_trace(
        
        go.Bar(
        name = f'{i}',
        x=X[mask].Date,
        y=X[mask].Rt,
        text=round(X[mask].Rt,3),
        textposition='inside',
        marker_color= colors_dict.get(i),
        
        showlegend=True))




                  
fig.update_layout(
    annotations=[
        
            dict(x=0.5,y=-0.15,
            showarrow=False,
            text="Данные : Яндекс <br> формула расчета из методических рекомендациях Роспотребнадзора.",
            xref="paper",
            yref="paper")],
                  title = "Коэффициент распространения (Rt) по 6 странам ,<br> с максимальным количеством новых заболеваний и Украине ",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.9,
                  width = 990, height = 600,template = 'gridon',
    
    
                 # xaxis_title='Коэффициент рассчитан, исходя из данных оперативного штаба и <br>' + 
                               #' формулы расчета в методических рекомендациях Роспотребнадзора. ',
    yaxis_title = ' '
)


fig.show()


In [13]:
print('Все страны по которым есть статистика:')
print('*'*40)
print(country_dict.items())
print('*'*40)
print('Рассматриваем страны: ')
for i in country_dict.keys():
    if country_dict.get(i) in country_towatch:
        print(i,country_dict.get(i))

# Украина № 162
# Весь мир № 31
# Турция № 159
#  Танзания № 155
my_country = country_dict[162]
print('Выбрали страну: ',my_country)


Все страны по которым есть статистика:
****************************************
dict_items([(0, 'Австралия'), (1, 'Австрия'), (2, 'Азербайджан'), (3, 'Албания'), (4, 'Алжир'), (5, 'Ангола'), (6, 'Андорра'), (7, 'Антигуа и Барбуда'), (8, 'Аргентина'), (9, 'Армения'), (10, 'Афганистан'), (11, 'Багамские острова'), (12, 'Бангладеш'), (13, 'Барбадос'), (14, 'Бахрейн'), (15, 'Беларусь'), (16, 'Белиз'), (17, 'Бельгия'), (18, 'Бенин'), (19, 'Болгария'), (20, 'Боливия'), (21, 'Босния и Герцеговина'), (22, 'Ботсвана'), (23, 'Бразилия'), (24, 'Бруней'), (25, 'Буркина-Фасо'), (26, 'Бурунди'), (27, 'Бутан'), (28, 'Великобритания'), (29, 'Венгрия'), (30, 'Венесуэла'), (31, 'Весь мир'), (32, 'Восточный Тимор'), (33, 'Вьетнам'), (34, 'Габон'), (35, 'Гаити'), (36, 'Гайана'), (37, 'Гамбия'), (38, 'Гана'), (39, 'Гватемала'), (40, 'Гвинея'), (41, 'Гвинея-Бисау'), (42, 'Германия'), (43, 'Гондурас'), (44, 'Гренада'), (45, 'Греция'), (46, 'Грузия'), (47, 'ДР Конго'), (48, 'Дания'), (49, 'Джибути'), (50, 'До

In [14]:
mask = X.Country ==my_country
#mask = (X.Date>datetime.date(2020,5,1)&(X.Region.isin(regions_towatch))
#mask = (X.Date>datetime.date(2020,5,1)&(X.Region==my_region)

fig = px.bar(X[mask], x='Date', 
             y="Day_confirmed",color ='Day_confirmed',
             title=f'{my_country}: '+'график количества новых заболевших COVID-19 по датам',
              color_continuous_scale= px.colors.sequential.BuPu,text = 'Day_confirmed',
             labels = {'Day_confirmed' : "Новые заболевшие",'Remaining_ill':'Текущие больные'})
fig.add_trace(
    go.Scatter(
        name='14-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA14_dayconfirmed'],
        mode="lines",
        
        line=go.scatter.Line(color="green"),
        showlegend=True)
)
fig.add_trace(
    go.Scatter(
        name = '7-дневная скользящая средняя',
        x=X[mask]['Date'],
        y=X[mask]['MA7_dayconfirmed'],
        mode="lines",
        
        line=go.scatter.Line(color="red"),
        showlegend=True)
)
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=1355,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=1572,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-90),
        dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Данные: Яндекс",
            xref="paper",
            yref="paper"),
        
                ],
    
    
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.10,legend_y = 0.95,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ' '
)


fig.show()


In [15]:
mask = X.Country ==my_country
#mask = (X.Date>datetime.date(2020,5,1))&(X.Region==my_region)
Y = X[mask]

Y['base'] = Y['Change_dayconf']*Y['Change_dayconf'].apply(lambda d:0 if d>0 else 1)
Y['Date']= pd.to_datetime(Y.Date)
fig = go.Figure()
fig.add_trace(
              go.Bar(x = Y.Date,y=Y.Change_dayconf,text = Y['Change_dayconf'],textposition = 'inside',
                
                #marker_color=Y['Change_dayconf'].apply(lambda d:'crimson' if d>0 else 'green'),
                 marker_color = Y['Change_dayconf'],marker_colorscale = 'Temps',marker_colorbar ={'tickmode':'auto',
                                    'title':{'text':'Изменение','side':'top'}}
                    )   
                )
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=-429,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=60),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title=f'{my_country}: динамика новых заболевших COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Данные: Яндекс',
    yaxis_title = ' '
)


fig.show()


In [16]:
mask = X.Country ==my_country
#mask = (X.Date>'2020-05-01')&(X.Region.isin(regions_towatch))

fig = px.bar(X[mask], x='Date', y="Remaining_ill",color ='Remaining_ill',
             title=f'{my_country}: '+'график количества текущих больных COVID-19 по датам',
              color_continuous_scale= px.colors.sequential.Purp,text = 'Remaining_ill',
            labels = {'Day_confirmed' : "Новые заболевшие",'Remaining_ill':'Текущие больные'})
fig.add_trace(
    go.Scatter(
        name='14-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA14_remaining_ill'],
        mode="lines",
        
        line=go.scatter.Line(color="green"),
        showlegend=True)
)

fig.add_trace(
    go.Scatter(
        name = '7-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA7_remaining_ill'],
        mode="lines",
        
        line=go.scatter.Line(color="red"),
        showlegend=True)
)
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=85824,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.15,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Текущие больные = количество выявленных - количество выздоровевших - количество умерших'+
                                '<br>Данные: Яндекс',
    yaxis_title = ' '
)


fig.show()


In [17]:
mask = X.Country ==my_country
#mask = (X.Date>'2020-05-01')&(X.Region==my_region)
Y = X[mask]

#Y['base'] = Y['Change_dayconf']*Y['Change_dayconf'].apply(lambda d:0 if d>0 else 1)
Y['Date']= pd.to_datetime(Y.Date)
fig = go.Figure()
fig.add_trace(
              go.Bar(x = Y.Date,y=Y.Change_remill,text = Y['Change_remill'],textposition = 'inside',
                
                #marker_color=Y['Change_remill'].apply(lambda d:'crimson' if d>0 else 'green'),
                marker_color = Y['Change_remill'],marker_colorscale = 'Temps',marker_colorbar ={'tickmode':'auto',
                                    'title':{'text':'Изменение','side':'top'}})     
                )
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=-3877,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=-20,
#             ay=40),
#         dict(
#             x='2020-05-20',
#             y=-2816,
#             xref="x",
#             yref="y",
#             text="20 мая",
#             showarrow=True,
#             arrowhead=2,
#             ax=-20,
#             ay=40),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title=f'{my_country}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Текущие больные = количество выявленных - количество выздоровевших - количество умерших'+
                               '<br>Данные : Яндекс',
    yaxis_title = ' '
)


fig.show()


In [18]:
mask = (X.Date>=datetime.date(2020,7,30))&(X.Country.isin(country_towatch))

def my_round(k):
    return round(k,3)

# #mask = (X.Date>='2020-06-04')

Z = X[mask].groupby(['Country','Date']).Rt.sum().apply(my_round).unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Country',axis = 1).values

# my_map= {'Москва':'2.Москва','Московская обл.':'3.Московская обл.','Нижегородская обл.':'5.Нижегородская обл.',
#                     'Россия':'1.Россия', 'Санкт-Петербург':'4.Санкт-Петербург'}
# Z.Region = Z.Region.map(my_map)

print(d)
Z.head()

[[0.997 1.421 1.607 1.15  0.734 0.633 0.673 0.984 1.513 1.51  1.166 0.814
  0.694 0.724 1.074 1.501 1.41  1.151 0.711 0.605 0.673 0.922 1.283 1.327]
 [0.942 1.036 1.196 1.162 1.156 1.016 0.991 1.074 1.031 1.128 1.144 1.031
  1.116 1.165 1.11  1.334 1.219 1.156 1.043 0.831 0.805 0.811 0.969 1.108]
 [1.012 1.125 1.125 1.129 1.11  0.983 0.976 1.007 1.054 1.153 1.172 1.082
  1.035 0.995 0.982 1.065 1.08  1.03  0.981 0.938 0.949 1.028 1.131 1.151]
 [2.671 1.13  0.966 0.575 0.894 1.467 2.18  3.627 1.489 0.808 0.498 0.615
  0.708 1.335 2.673 1.511 1.323 0.845 0.947 1.078 1.732 2.693 1.24  1.022]
 [0.99  1.081 1.085 0.972 0.837 0.769 0.769 0.893 1.044 1.086 1.087 0.979
  0.874 0.879 0.924 1.038 1.103 1.032 0.928 0.773 0.767 0.829 0.973 1.092]]


Date,Country,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,...,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22
0,Бразилия,0.997,1.421,1.607,1.150,0.734,0.633,0.673,0.984,1.513,...,1.074,1.501,1.410,1.151,0.711,0.605,0.673,0.922,1.283,1.327
1,Великобритания,0.942,1.036,1.196,1.162,1.156,1.016,0.991,1.074,1.031,...,1.110,1.334,1.219,1.156,1.043,0.831,0.805,0.811,0.969,1.108
2,Индия,1.012,1.125,1.125,1.129,1.110,0.983,0.976,1.007,1.054,...,0.982,1.065,1.080,1.030,0.981,0.938,0.949,1.028,1.131,1.151
3,Испания,2.671,1.130,0.966,0.575,0.894,1.467,2.180,3.627,1.489,...,2.673,1.511,1.323,0.845,0.947,1.078,1.732,2.693,1.240,1.022
4,США,0.990,1.081,1.085,0.972,0.837,0.769,0.769,0.893,1.044,...,0.924,1.038,1.103,1.032,0.928,0.773,0.767,0.829,0.973,1.092


In [19]:
x1=list(Z.columns[1:])
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
y=list(Z.Country)
print(list(x1))
print(x2)

[datetime.date(2020, 7, 30), datetime.date(2020, 7, 31), datetime.date(2020, 8, 1), datetime.date(2020, 8, 2), datetime.date(2020, 8, 3), datetime.date(2020, 8, 4), datetime.date(2020, 8, 5), datetime.date(2020, 8, 6), datetime.date(2020, 8, 7), datetime.date(2020, 8, 8), datetime.date(2020, 8, 9), datetime.date(2020, 8, 10), datetime.date(2020, 8, 11), datetime.date(2020, 8, 12), datetime.date(2020, 8, 13), datetime.date(2020, 8, 14), datetime.date(2020, 8, 15), datetime.date(2020, 8, 16), datetime.date(2020, 8, 17), datetime.date(2020, 8, 18), datetime.date(2020, 8, 19), datetime.date(2020, 8, 20), datetime.date(2020, 8, 21), datetime.date(2020, 8, 22)]
['30.7.2020', '31.7.2020', '1.8.2020', '2.8.2020', '3.8.2020', '4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020', '14.8.2020', '15.8.2020', '16.8.2020', '17.8.2020', '18.8.2020', '19.8.2020', '20.8.2020', '21.8.2020', '22.8.2020']


In [20]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,    
    
                  xaxis_title='Динамика коэффициента распространения Rt по странам',
               #   yaxis_title = 'Регион '
)

fig.show()



In [21]:
mask = (X.Date>=datetime.date(2020,7,26))&(X.Country.isin(country_towatch))


# #mask = (X.Date>='2020-06-04')

Z = X[mask].groupby(['Country','Date']).Rt2.sum().apply(my_round).unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Country',axis = 1).values

# my_map= {'Москва':'2.Москва','Московская обл.':'3.Московская обл.','Нижегородская обл.':'5.Нижегородская обл.',
#                     'Россия':'1.Россия', 'Санкт-Петербург':'4.Санкт-Петербург'}
# Z.Region = Z.Region.map(my_map)

print(d)
Z.head()

[[1.098 1.092 1.08  1.07  1.068 1.074 1.081 1.083 1.076 1.067 1.058 1.057
  1.063 1.068 1.07  1.066 1.059 1.053 1.053 1.059 1.063 1.064 1.059 1.051
  1.045 1.043 1.046 1.048]
 [1.009 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.011 1.011 1.011 1.011
  1.011 1.011 1.012 1.012 1.012 1.012 1.013 1.014 1.014 1.015 1.015 1.014
  1.013 1.013 1.012 1.013]
 [1.157 1.157 1.153 1.149 1.144 1.143 1.142 1.142 1.141 1.136 1.13  1.126
  1.124 1.125 1.126 1.125 1.122 1.119 1.115 1.113 1.112 1.11  1.107 1.102
  1.098 1.097 1.097 1.099]
 [1.027 1.026 1.026 1.029 1.037 1.038 1.037 1.033 1.031 1.035 1.043 1.056
  1.06  1.057 1.049 1.042 1.037 1.04  1.051 1.056 1.058 1.056 1.054 1.054
  1.062 1.078 1.08  1.079]
 [1.07  1.068 1.065 1.062 1.061 1.061 1.061 1.06  1.057 1.052 1.048 1.046
  1.046 1.047 1.047 1.046 1.044 1.042 1.041 1.041 1.042 1.041 1.04  1.038
  1.035 1.033 1.032 1.033]]


Date,Country,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,...,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22
0,Бразилия,1.098,1.092,1.080,1.070,1.068,1.074,1.081,1.083,1.076,...,1.053,1.059,1.063,1.064,1.059,1.051,1.045,1.043,1.046,1.048
1,Великобритания,1.009,1.010,1.010,1.010,1.010,1.010,1.010,1.010,1.011,...,1.013,1.014,1.014,1.015,1.015,1.014,1.013,1.013,1.012,1.013
2,Индия,1.157,1.157,1.153,1.149,1.144,1.143,1.142,1.142,1.141,...,1.115,1.113,1.112,1.110,1.107,1.102,1.098,1.097,1.097,1.099
3,Испания,1.027,1.026,1.026,1.029,1.037,1.038,1.037,1.033,1.031,...,1.051,1.056,1.058,1.056,1.054,1.054,1.062,1.078,1.080,1.079
4,США,1.070,1.068,1.065,1.062,1.061,1.061,1.061,1.060,1.057,...,1.041,1.041,1.042,1.041,1.040,1.038,1.035,1.033,1.032,1.033


In [22]:
x1=list(Z.columns[1:])
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
y=list(Z.Country)
print(list(x1))
print(x2)

[datetime.date(2020, 7, 26), datetime.date(2020, 7, 27), datetime.date(2020, 7, 28), datetime.date(2020, 7, 29), datetime.date(2020, 7, 30), datetime.date(2020, 7, 31), datetime.date(2020, 8, 1), datetime.date(2020, 8, 2), datetime.date(2020, 8, 3), datetime.date(2020, 8, 4), datetime.date(2020, 8, 5), datetime.date(2020, 8, 6), datetime.date(2020, 8, 7), datetime.date(2020, 8, 8), datetime.date(2020, 8, 9), datetime.date(2020, 8, 10), datetime.date(2020, 8, 11), datetime.date(2020, 8, 12), datetime.date(2020, 8, 13), datetime.date(2020, 8, 14), datetime.date(2020, 8, 15), datetime.date(2020, 8, 16), datetime.date(2020, 8, 17), datetime.date(2020, 8, 18), datetime.date(2020, 8, 19), datetime.date(2020, 8, 20), datetime.date(2020, 8, 21), datetime.date(2020, 8, 22)]
['26.7.2020', '27.7.2020', '28.7.2020', '29.7.2020', '30.7.2020', '31.7.2020', '1.8.2020', '2.8.2020', '3.8.2020', '4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020'

In [23]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,    
    
                  xaxis_title='Динамика коэффициента распространения Rt2 по странам',
               #   yaxis_title = 'Регион '
)

fig.show()


In [24]:
mask = (X.Date>=datetime.date(2020,7,26))&(X.Country.isin(country_towatch))

Z = X[mask].groupby(['Country','Date'])['Day_confirmed'].sum().unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Country',axis = 1).values
Z.head()

Date,Country,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,...,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22
0,Бразилия,24578,23284,40816,69074,57837,52383,45392,25800,16641,...,60091,50644,41576,23101,19373,47784,49298,45323,30355,50032
1,Великобритания,750,688,553,802,847,883,769,747,942,...,1148,1461,1042,1111,721,1113,831,1195,1060,1332
2,Индия,49981,44457,51596,50294,52783,61242,54735,52972,52050,...,64553,64732,64030,57711,55018,64572,69672,68900,69876,69239
3,Испания,0,6361,1828,2031,2789,3092,0,0,8532,...,7550,5479,0,0,16269,5114,6671,7039,8148,0
4,США,54953,56414,65869,70776,68033,67092,58485,47580,45368,...,51547,64294,47913,42048,35112,44091,47408,44023,48693,44572


In [25]:



x1=list(Z.columns[1:])
y=list(Z.Country)
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
print(x2)


['26.7.2020', '27.7.2020', '28.7.2020', '29.7.2020', '30.7.2020', '31.7.2020', '1.8.2020', '2.8.2020', '3.8.2020', '4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020', '14.8.2020', '15.8.2020', '16.8.2020', '17.8.2020', '18.8.2020', '19.8.2020', '20.8.2020', '21.8.2020', '22.8.2020']


In [26]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,
    
    
                  xaxis_title='Динамика новых заболевших по странам',
               #   yaxis_title = 'Регион '
)

fig.show()

In [27]:
mask = (X.Date==myday)&(X.Country != 'Весь мир')
#mask = (X.Date==myday)
#Делаем выборку по доле регина в общих случаях выявления заболеваний по России
df=X[mask].sort_values(by=['Rate_conf'],ascending = False)
#Далее заменяем все имена если регион набрал менее 1% в общих случаях по России
df2 = X[mask].reset_index(drop=True)
print(len(df2))

for i in range(len(df2)):
    if df2.loc[i,'Rate_conf']<1:
        df2.loc[i,'Country']='Страны менее 1%'
df.head(15)

187


,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths,Rt,Rt2,Rate_conf,Rate_ill,Country,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
28247,2020-08-22,5667112,1985484,176353,44572,20428,983,1.092,1.033,24.423489,46.813168,США,3505275,43706.714286,47798.785714,3.440903e+06,3.367124e+06,-4121,23161
5135,2020-08-22,3582362,2913966,114250,50032,58408,892,1.327,1.048,15.438865,7.400655,Бразилия,554146,37895.142857,40710.714286,5.545766e+05,5.623751e+05,19677,-9268
12625,2020-08-22,3044940,2280566,56706,69239,57989,912,1.151,1.099,13.122744,9.450951,Индия,707668,64998.285714,63709.285714,6.871550e+05,6.706991e+05,-637,10338
27605,2020-08-22,949531,765754,16268,4860,6115,120,0.988,1.021,4.092183,2.237093,Россия,167509,4817.571429,4926.285714,1.714154e+05,1.740157e+05,22,-1375
39375,2020-08-22,607045,504127,12987,3707,4025,144,1.146,1.023,2.616175,1.201034,ЮАР,89931,3341.714286,3846.928571,9.434957e+04,1.088698e+05,309,-462
26749,2020-08-22,576067,380730,27245,9008,0,211,1.372,1.062,2.482669,2.244879,Перу,168092,8538.714286,8013.714286,1.482507e+05,1.410936e+05,369,8797
22255,2020-08-22,556216,453104,60254,6482,4760,644,1.257,1.043,2.397118,0.572371,Мексика,42858,5500.285714,5736.714286,4.076629e+04,4.173543e+04,554,1078
16905,2020-08-22,522138,348940,16568,0,0,0,0.750,1.090,2.250252,2.091804,Колумбия,156630,9349.857143,10376.285714,1.597593e+05,1.631893e+05,-8419,0
37235,2020-08-22,395708,369730,10792,1939,1833,69,1.016,1.016,1.705378,0.202810,Чили,15186,1686.571429,1763.214286,1.568043e+04,1.627243e+04,19,37
14123,2020-08-22,386054,150376,28838,0,0,0,1.022,1.079,1.663773,2.762361,Испания,206840,6177.285714,5120.857143,1.904459e+05,1.712221e+05,-8148,0


In [28]:



fig = px.pie(df2, values='Confirmed', names='Country',color_discrete_sequence=px.colors.sequential.Rainbow,
            title=f'Всего выявленных случаев заболевания COVID-19 в мире: {df2.Confirmed.sum()}<br>'+
            f'(на дату: {myday})'
            ,
            width = 990, height=1000)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
        
                #  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  margin = {'t':150, 'b':0},
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.98,
                     
)

fig.show()



In [29]:
mask = (X.Date==myday)&(X.Country !='Весь мир')
#mask = (X.Date==myday)
#Делаем выборку по доле регина в общих случаях выявления заболеваний по России
df=X[mask].sort_values(by=['Rate_ill'],ascending = False)
#Далее заменяем все имена если регион набрал менее 1% в общих случаях по России
df2 = X[mask].reset_index(drop=True)
print(len(df2))

for i in range(len(df2)):
    if df2.loc[i,'Rate_ill']<1:
        df2.loc[i,'Country']='Страны менее 1%'
df.head(20)

187


,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths,Rt,Rt2,Rate_conf,Rate_ill,Country,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
28247,2020-08-22,5667112,1985484,176353,44572,20428,983,1.092,1.033,24.423489,46.813168,США,3505275,43706.714286,47798.785714,3.440903e+06,3.367124e+06,-4121,23161
12625,2020-08-22,3044940,2280566,56706,69239,57989,912,1.151,1.099,13.122744,9.450951,Индия,707668,64998.285714,63709.285714,6.871550e+05,6.706991e+05,-637,10338
5135,2020-08-22,3582362,2913966,114250,50032,58408,892,1.327,1.048,15.438865,7.400655,Бразилия,554146,37895.142857,40710.714286,5.545766e+05,5.623751e+05,19677,-9268
6205,2020-08-22,326595,1547,41509,1332,3,18,1.108,1.013,1.407523,3.786681,Великобритания,283539,1051.857143,1081.000000,2.802444e+05,2.765559e+05,272,1311
14123,2020-08-22,386054,150376,28838,0,0,0,1.022,1.079,1.663773,2.762361,Испания,206840,6177.285714,5120.857143,1.904459e+05,1.712221e+05,-8148,0
26749,2020-08-22,576067,380730,27245,9008,0,211,1.372,1.062,2.482669,2.244879,Перу,168092,8538.714286,8013.714286,1.482507e+05,1.410936e+05,369,8797
27605,2020-08-22,949531,765754,16268,4860,6115,120,0.988,1.021,4.092183,2.237093,Россия,167509,4817.571429,4926.285714,1.714154e+05,1.740157e+05,22,-1375
35951,2020-08-22,275562,85102,30517,3602,121,9,2.766,1.041,1.187586,2.136049,Франция,159943,3228.142857,2882.428571,1.460741e+05,1.378982e+05,-11824,3472
16905,2020-08-22,522138,348940,16568,0,0,0,0.750,1.090,2.250252,2.091804,Колумбия,156630,9349.857143,10376.285714,1.597593e+05,1.631893e+05,-8419,0
2781,2020-08-22,292625,175567,3907,2265,2952,46,0.983,1.039,1.261123,1.511139,Бангладеш,113151,2585.714286,2679.428571,1.146190e+05,1.120265e+05,-136,-733


In [30]:


fig = px.pie(df2, values='Remaining_ill', names='Country',color_discrete_sequence=px.colors.sequential.Rainbow,
            title=f'Всего текущих больных COVID-19 в мире: {df2.Remaining_ill.sum()}<br>'+
            f'(на дату: {myday})'
             ,
            width = 990, height=1100)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
        
                #  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  margin = {'t':120, 'b':0},
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.98,
                     
)

fig.show()


In [31]:
covid_rates = pd.DataFrame()
mask = (X.Date>=datetime.date(2020,8,2))&(X.Date<=datetime.date(2020,8,9))&(X.Country.isin(country_towatch))&(X.Country!='Весь мир')
                                      
box_cases1 = X[mask].groupby(['Country','Date'])['Day_confirmed'].sum()
box_cases2 = X[mask].groupby(['Country','Date'])['Day_recovered'].sum()
for i in country_dict.keys():
    if country_dict.get(i) in country_towatch:
        print(i,country_dict.get(i))

23 Бразилия
28 Великобритания
58 Индия
65 Испания
131 США


In [32]:
new_df1=box_cases1.unstack().T
new_df1.tail(20)

Country,Бразилия,Великобритания,Индия,Испания,США
Date,,,,,
2020-08-02,25800,747,52972,0,47580
2020-08-03,16641,942,52050,8532,45368
2020-08-04,51603,675,52509,5760,57540
2020-08-05,57152,906,56282,2953,52810
2020-08-06,53139,964,62538,4088,59692
2020-08-07,50230,900,61537,4507,58173
2020-08-08,49970,765,64399,0,56174
2020-08-09,23010,1113,62064,0,46935


In [33]:
color_dict = {
                "США": "red",
                "Бразилия": "magenta",
                "Индия": "goldenrod",
                "Великобритания": "darkblue",
                "Россия": "green",
                 }

cols = list(new_df1.columns)
#cols = cols[:-1]

fig = go.Figure()
for i in cols:
    fig.add_trace(
    go.Box(x=new_df1[i],name = i,boxmean = True,marker_color = color_dict.get(i)))

fig.update_layout(title_text="Статистика по новым заболевшим за неделю",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "top",
                  xaxis_title='Новые заболевшие', yaxis_title='Страны',width = 990, height = 600)
fig.show()

In [34]:
new_df2=box_cases2.unstack().T
new_df2.tail(20)

Country,Бразилия,Великобритания,Индия,Испания,США
Date,,,,,
2020-08-02,24894,3,40574,0,6804
2020-08-03,36100,1,44306,0,44757
2020-08-04,58508,0,51706,0,15533
2020-08-05,32877,1,46121,0,48872
2020-08-06,40181,1,49769,0,20773
2020-08-07,41757,0,48900,0,25246
2020-08-08,49238,2,53879,0,19248
2020-08-09,35446,2,54859,0,13746


In [35]:

cols = list(new_df2.columns)
#cols = cols[:-1]

fig = go.Figure()
for i in cols:
    fig.add_trace(
    go.Box(x=new_df2[i],name = i,boxmean=True,marker_color = color_dict.get(i)))

fig.update_layout(title_text="Статистика по выздоровевшим за неделю",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "top",
                  xaxis_title='Выздоровевшие', yaxis_title='Регионы',width = 990, height = 600)
fig.show()

In [36]:
from plotly.subplots import make_subplots

# fig = tools.make_subplots(rows = 1, cols = 2, specs = [[{}, {}]],
#                           horizontal_spacing = 0.05)


fig = make_subplots(rows=2, cols=1,specs = [[{}],[{}]],vertical_spacing = 0.03,shared_xaxes=True,
                  subplot_titles=("Box-график распределения по новым заболевшим(в день) за неделю:",
                                 "Box-график распределения по выздоровевшим(в день) за неделю:"))
#subplot_titles=("Статистика по новым заболевшим за последние 2 недели","Статистика по выздоровевшим за последние 2 недели")

for i in cols:
    fig.add_trace(
    go.Box(x=new_df1[i],name = i,boxmean = True,marker_color = color_dict.get(i)),row=1,col=1)

for i in cols:
    fig.add_trace(
    go.Box(x=new_df2[i],name = i,boxmean=True,marker_color = color_dict.get(i)),row=2,col=1)


fig.update_layout(
#     annotations=[
#         dict(
#             x=0.5,
#             y=-0.15,
#             showarrow=False,
#             text="Ветикальная пунктирная линия - среднее, Вертикальная сплошная - медиана распоеделения",
#             xref="paper",
#             yref="paper"),
        
#                 ],
    
    
                  width = 990, height = 1200,
    
    
                 showlegend=False,template ='ggplot2'
)

#fig.update_layout(height=1000, width=800,showlegend=False)
fig.show()